# Runs the script to generate windows across all exons (CDS, UTRs)

In [1]:
import glob
import os
from qtools import Submitter
from tqdm import tnrange, tqdm_notebook

In [2]:
input_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/sailor_outputs_groups_deep/combined_outputs_w_cov_info/'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/window_outputs_deep/'

In [3]:
all_annotated = sorted(glob.glob(os.path.join(input_dir, 'RPS2-STAMP_possorted_genome_bam_MD.fx.annotated')))
print(len(all_annotated))
all_annotated[:3]

1


['/home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/sailor_outputs_groups_deep/combined_outputs_w_cov_info/RPS2-STAMP_possorted_genome_bam_MD.fx.annotated']

# Write the commands to score all exon edits.
- 2020-12-04 let's just look at CDS for now.

In [4]:
# bg_edits_file = '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/outputs/combined_outputs_w_cov_info/ApoControl-1000_S21_L002_R1_001.fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted_a0_b0_e0.01.bed'

chrom_sizes_file = '/projects/ps-yeolab3/bay001/annotations/hg19/hg19.chrom.sizes'
gtfdb_file = '/projects/ps-yeolab3/bay001/annotations/hg19/gencode_v19/gencode.v19.annotation.gtf.db'
genome_fa = '/projects/ps-yeolab3/bay001/annotations/hg19/hg19.fa'

cds_file = '/projects/ps-yeolab3/bay001/annotations/hg19/gencode_v19/hg19_v19_cds.bed'
three_prime_utr_file = '/projects/ps-yeolab3/bay001/annotations/hg19/gencode_v19/hg19_v19_three_prime_utrs.bed'
five_prime_utr_file = '/projects/ps-yeolab3/bay001/annotations/hg19/gencode_v19/hg19_v19_five_prime_utrs.bed'

cmds = []
for annotated in all_annotated:
    output_file = os.path.join(output_dir, os.path.basename(annotated) + '.exons.txt')
    output_file_summed = os.path.join(output_dir, os.path.basename(annotated) + '.exons.merged.txt')
    
    pos_bw = os.path.splitext(annotated)[0][:-3] + '.fwd.sorted.rmdup.readfiltered.sorted.bw' # [:-3] to remove the ".fx"
    neg_bw = os.path.splitext(annotated)[0][:-3] + '.rev.sorted.rmdup.readfiltered.sorted.bw'
    if not os.path.exists(output_file) and (os.path.exists(pos_bw) and os.path.exists(neg_bw)):
        cmd = 'module load python3essential;'
        cmd += '/home/bay001/projects/kris_apobec_20200121/scripts/score_edits_total_exon_coverage_sc.py '
        cmd += '--conf 0 ' 
        cmd += '--gtfdb {} '.format(gtfdb_file)
        cmd += '--chrom_sizes_file {} '.format(chrom_sizes_file)
        cmd += '--pos_bw {} '.format(pos_bw)
        cmd += '--neg_bw {} '.format(neg_bw)
        cmd += '--annotated_edits_file {} '.format(annotated)
        # cmd += '--bg_edits_file {} '.format(bg_edits_file)
        cmd += '--genome_fa {} '.format(genome_fa)
        cmd += '--output_file {} '.format(output_file)
        cmd += '--output_file_summed {} '.format(output_file_summed)
        cmd += '--three_prime_utr_file {} '.format(three_prime_utr_file)
        cmd += '--five_prime_utr_file {} '.format(five_prime_utr_file)
        cmd += '--cds_file {} '.format(cds_file)
        cmds.append(cmd)
    elif not os.path.exists(pos_bw):
        print("{} does not exist.".format(pos_bw))
    elif not os.path.exists(neg_bw):
        print("{} does not exist.".format(neg_bw))
print(len(cmds))

1


In [5]:
cmds

['module load python3essential;/home/bay001/projects/kris_apobec_20200121/scripts/score_edits_total_exon_coverage_sc.py --conf 0 --gtfdb /projects/ps-yeolab3/bay001/annotations/hg19/gencode_v19/gencode.v19.annotation.gtf.db --chrom_sizes_file /projects/ps-yeolab3/bay001/annotations/hg19/hg19.chrom.sizes --pos_bw /home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/sailor_outputs_groups_deep/combined_outputs_w_cov_info/RPS2-STAMP_possorted_genome_bam_MD.fwd.sorted.rmdup.readfiltered.sorted.bw --neg_bw /home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/sailor_outputs_groups_deep/combined_outputs_w_cov_info/RPS2-STAMP_possorted_genome_bam_MD.rev.sorted.rmdup.readfiltered.sorted.bw --annotated_edits_file /home/bay001/projects/kris_apobec_20200121/permanent_data2/07_scRNA_groups/sailor_outputs_groups_deep/combined_outputs_w_cov_info/RPS2-STAMP_possorted_genome_bam_MD.fx.annotated --genome_fa /projects/ps-yeolab3/bay001/annotations/hg19/hg19.fa -

In [6]:
# Submitter(commands=cmds, job_name='score_exon_edits', array=True, nodes=1, ppn=4, submit=True, walltime='48:00:00')